In [30]:
import pandas as pd
import numpy as np
from datetime import timedelta
from pandas_ta.volatility import atr
from pandas_ta.trend import adx
import matplotlib.pyplot as plt

import plotly.graph_objects as go

import os
import pickle
from config import BASE_DIR
print(os.getcwd())
os.chdir(BASE_DIR)
import scripts.functions as fn 
from scripts.indicators import resample, zigzag, supertrend, get_pivots_alert, technical_summary


C:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app


In [31]:
#klines_file  = './backtest/klines/0m01/Alts_AAVEUSDT_0m01_2024-05-28_2025-05-28.DataFrame'
#klines_file  = './backtest/klines/0m01/Alts_GMXUSDT_0m01_2024-05-28_2025-05-28.DataFrame'
#klines_file  = './backtest/klines/0m01/Alts_GALAUSDT_0m01_2024-05-28_2025-05-28.DataFrame'
#klines_file  = './backtest/klines/0m01/Alts_AVAXUSDT_0m01_2024-05-28_2025-05-28.DataFrame'

#klines_file  = './backtest/klines/0m01/Completo_BTCUSDT_0m01_2021-01-01_2023-07-31.DataFrame'
#klines_file  = './backtest/klines/0m01/Completo_BNBUSDT_0m01_2021-01-01_2023-07-31.DataFrame'
klines_file  = './backtest/klines/0m01/Completo_ETHUSDT_0m01_2021-01-01_2023-07-31.DataFrame'

with open(klines_file, 'rb') as file:
    df = pickle.load(file)

print('Total de velas - Iniciales',len(df))
df = df[0:100000]
df.reset_index(inplace=True, drop=True)
print('Total de velas - Ajustadas',len(df))

#dfBIG = resample(df,periods=BIG)
#dfBIG['atr'] = atr(dfBIG['high'],dfBIG['low'],dfBIG['close'],length=14)
#dfBIG['atr_ma'] = dfBIG['atr'].rolling(window=14).mean()
#dfBIG['atr_rel'] = dfBIG['atr']/dfBIG['atr_ma'] - 1
#dfBIG['vol_ma'] = dfBIG['volume'].rolling(window=100).mean()
#dfBIG['vol_rel'] = dfBIG['volume']/dfBIG['vol_ma'] - 1  
#dfBIG = supertrend(dfBIG)
#dfBIG = zigzag(dfBIG)
#
#dfBIG.drop(columns=['vol_ma','volume','atr','atr_ma'],inplace=True)



Total de velas - Iniciales 1657225
Total de velas - Ajustadas 100000


In [32]:
print('Inicia el proceso')
minutes_x_kline = 60
# --- Parámetros ---
MAX_BIGMIN_CANDLES = 200 # Número máximo de velas de BIGmin a mantener

# --- Inicialización ---
# DataFrame para las velas de BIG minutos que se van formando
dfBIG_cols = ['datetime', 'open', 'high', 'low', 'close', 'volume']
dfBIG = pd.DataFrame(columns=dfBIG_cols)

last_BIGmin_candle_start_time = None


open_pos = False
alert = {}
trade_columns = ['open','side','buy','sell','close','tp','sl','adx','adx_ma']
trades = pd.DataFrame(columns=trade_columns)
trade = {}
tp = None
sl = None
total_velas = len(df)
progreso = 0
kline_minutes = 0
# --- Iteración sobre el DataFrame de 1 minuto ---
for row_1min in df.itertuples(index=False): # index=False para no incluir el índice de df_1min
    progreso+=1
    progreso_perc = (progreso/total_velas)*100
    print(f'Procesando: {progreso_perc:.2f}% ',len(trades),'   ',end="\r")
    current_1min_timestamp = row_1min.datetime # Accedemos a la columna datetime
    current_1min_data = row_1min # El namedtuple entero

    this_BIGmin_slot_start_time = current_1min_timestamp.floor(f'{minutes_x_kline}T')

    is_new_BIGmin_candle = False
    # Comparamos con la columna 'datetime' de la última fila de dfBIG
    if len(dfBIG)==0 or this_BIGmin_slot_start_time > dfBIG['datetime'].iloc[-1]:
        is_new_BIGmin_candle = True

    if is_new_BIGmin_candle:
        new_BIGmin_candle_data = {
            'datetime': this_BIGmin_slot_start_time,
            'open': current_1min_data.open, # Acceso a campos del namedtuple
            'high': current_1min_data.high,
            'low': current_1min_data.low,
            'close': current_1min_data.close,
            'volume': current_1min_data.volume
        }
        
        # Añadir como una nueva fila. pd.concat es más robusto aquí.
        new_row_df = pd.DataFrame([new_BIGmin_candle_data])
        dfBIG = pd.concat([dfBIG, new_row_df], ignore_index=True) # ignore_index para mantener el índice numérico secuencial

        if len(dfBIG) > MAX_BIGMIN_CANDLES:
            dfBIG = dfBIG.iloc[-MAX_BIGMIN_CANDLES:].reset_index(drop=True) # reset_index para mantener secuencia 0..N-1
        kline_minutes = 1
    else:
        # Actualizar la última fila de dfBIG usando su índice numérico (.iloc[-1] o len(dfBIG)-1)
        last_row_index = len(dfBIG) - 1
        dfBIG.loc[last_row_index, 'high'] = max(dfBIG.loc[last_row_index, 'high'], current_1min_data.high)
        dfBIG.loc[last_row_index, 'low'] = min(dfBIG.loc[last_row_index, 'low'], current_1min_data.low)
        dfBIG.loc[last_row_index, 'close'] = current_1min_data.close
        dfBIG.loc[last_row_index, 'volume'] += current_1min_data.volume
        # 'datetime' y 'open' no cambian
        kline_minutes +=1

    SR_THRESHOLD = 0.1
    SR_THRESHOLD_STRONG = 0.5
    if len(dfBIG) >= MAX_BIGMIN_CANDLES:

        if not open_pos:
            if kline_minutes==minutes_x_kline:
                alert = get_pivots_alert(dfBIG[-150:],threshold=1.5)
                #brief = technical_summary(dfBIG[-150:])
                #sr = brief['sentiment_ratio']

                if alert['alert']!=0:
                    if True:
                    #if (alert['side'] > 0 and sr>SR_THRESHOLD and sr<SR_THRESHOLD_STRONG) or \
                    #   (alert['side'] < 0 and sr<-SR_THRESHOLD and sr>-SR_THRESHOLD_STRONG) :
                        open_pos = True
                        trade = {'open': current_1min_timestamp,
                                'close': None,
                                'side': alert['side'],
                                'buy': row_1min.close if alert['side'] > 0 else None,
                                'sell': row_1min.close if alert['side'] < 0 else None,
                                'sl': alert['sl1'],
                                'tp': alert['tp1'],
                                'adx': alert['adx'],
                                'adx_ma': alert['adx_ma'],
                                }
                        tp = alert['tp1']
                        sl = alert['sl1']
                
        else: #open_pos = True
            price = row_1min.close
            high = row_1min.high
            low = row_1min.low

            if trade['side']>0: #LONG
                if high >= tp:
                    trade['sell'] = tp
                    trade['close'] = current_1min_timestamp
                    open_pos = False
                    trades.loc[len(trades)] = trade
                    trade = {}
                elif low <= sl:
                    trade['sell'] = sl
                    trade['close'] = current_1min_timestamp
                    open_pos = False
                    trades.loc[len(trades)] = trade
                    trade = {}                    
  
            else: #SHORT
                if low <= tp:
                    trade['buy'] = tp
                    trade['close'] = current_1min_timestamp
                    open_pos = False
                    trades.loc[len(trades)] = trade
                    trade = {}
                if high >= sl:
                    trade['buy'] = sl
                    trade['close'] = current_1min_timestamp
                    open_pos = False
                    trades.loc[len(trades)] = trade
                    trade = {}
      
    




Inicia el proceso


In [33]:
trades['result'] = (trades['sell']/trades['buy'] - 1 ) *100
trades['adx_force'] = np.where((trades['adx']>20) & (trades['adx']>trades['adx_ma']),1,0)

long_trades = trades[trades['side']> 0 ]
short_trades = trades[trades['side']< 0 ]

tot_trades = long_trades['result'].count()
trades_win = long_trades[long_trades['result']> 0 ]['result'].count()
result = long_trades['result'].sum()

print('LONG Trades:',tot_trades)
print(' Win',round((trades_win/tot_trades)*100,2),'%')
print(' Resultado',round(result,2),'%')

tot_trades = short_trades['result'].count()
trades_win = short_trades[short_trades['result']> 0 ]['result'].count()
result = short_trades['result'].sum()

print('SHORT Trades:',tot_trades)
print(' Win',round((trades_win/tot_trades)*100,2),'%')
print(' Resultado',round(result,2),'%')

#print('Trades Positivos')
#print(trades[trades['result']>0][['adx','adx_force']].describe())
#print('Trades Negativos')
#print(trades[trades['result']<0][['adx','adx_force']].describe())

LONG Trades: 2
 Win 50.0 %
 Resultado 2.3 %
SHORT Trades: 1
 Win 0.0 %
 Resultado -1.7 %


LONG Trades: 8
 Win 37.5 %
 Resultado 1.82 %
SHORT Trades: 9
 Win 44.44 %
 Resultado 3.47 %

In [34]:
df_to_show = resample(df,periods=minutes_x_kline)

df_to_show['sl'] = None
df_to_show['tp'] = None
df_to_show['pnl'] = None
df_to_show['price'] = df_to_show['close']
df_events = pd.DataFrame(columns=['datetime','buy','sell'])
pnl = 0
for i,trade in trades.iterrows():
    if trade['side']>0:
        df_events.loc[len(df_events)] = {'datetime': trade['open'],'buy': trade['buy'],'sell': None,}
        df_events.loc[len(df_events)] = {'datetime': trade['close'],'buy': None,'sell': trade['sell'],}
    else:
        df_events.loc[len(df_events)] = {'datetime': trade['close'],'buy': trade['buy'],'sell': None,}
        df_events.loc[len(df_events)] = {'datetime': trade['open'],'buy': None,'sell': trade['sell'],}
    pnl += trade['result']
    df_to_show['sl'] = np.where((df_to_show['datetime']>=trade['open']) & (df_to_show['datetime']<=trade['close']), trade['sl'], df_to_show['sl'])
    df_to_show['tp'] = np.where((df_to_show['datetime']>=trade['open']) & (df_to_show['datetime']<=trade['close']), trade['tp'], df_to_show['tp'])
    df_to_show['pnl'] = np.where((df_to_show['datetime']>=trade['close']) , pnl, df_to_show['pnl'])
indicators = [
     {'col': 'sl','name': 'SL','color': 'red','row': 1, 'mode':'markers',},
     {'col': 'tp','name': 'TP','color': 'green','row': 1, 'mode':'markers',},
        ]

indicators_out = [
     #{'col': 'atr_rel','name': 'ATR','color': 'yellow','row': 1, 'mode':'bars',},
        ]

#https://plotly.com/python/reference/scatter/#scatter-marker-symbol
events = [
       {'df':df_events,'col':'buy','name': 'BUY','color': 'green','symbol': 'triangle-up' }, 
       {'df':df_events,'col':'sell','name': 'SELL','color': 'red','symbol': 'triangle-down' }, 
     ]


fig = fn.ohlc_chart(df_to_show,show_pnl=True,indicators=indicators,indicators_out=indicators_out,events=events,) 

fig.show()